In [ ]:
'''
Pipeline for RGS spectrum extraction.

Reference:
https://www.cosmos.esa.int/web/xmm-newton/sas-thread-rgs

Object:
SDSSTG5742

'''

In [1]:
import os
from glob import glob

0 Define path

In [13]:
homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/SDSSTG5742/raw/SDSSTG5742'

os.chdir(homepath)

1 Initialize steps

In [14]:
f = open('cifbuild.sh', 'w')
f.write(
f'''
export SAS_ODF={homepath}
cifbuild 2>&1 | tee cifbuild.log

'''
)
f.close()




In [5]:
f = open('odfingest.sh', 'w')
f.write(f'''
export SAS_CCF={homepath}/ccf.cif
export SAS_ODF={homepath}
odfingest odfdir=$SAS_ODF outdir=$SAS_ODF 2>&1 | tee odfingest.log
''')
f.close()

3 Set SAS env

In [10]:

f = open('set_sas.txt', 'w')
sasfile= glob('*SUM.SAS')[0] 
f.write(f'''
export SAS_CCF={homepath}/ccf.cif
export SAS_ODF={homepath}/{sasfile}
sasversion
''')
f.close()

## source set_sas.txt

4 run rgsproc

In [17]:
f = open('rgsproc.sh', 'w')

f.write(
'''rgsproc withsrc=true srcstyle=radec srcra=200.06 srcdec=33.1436 withbackgroundmodel=yes witharffile=true 2>&1 | tee rgsproc_1st.log 
''')

f.close()

5 Display the dispersion vs cross dispersion and dispersion vs energy images and overlay the selected region masks:

In [12]:
evenli_file = glob(f'*5EVENLI*FIT')[0]
srcli_file = glob(f'*5SRCLI*FIT')[0]


f = open('inspect.sh', 'w')
f.write(
f'''evselect table='{evenli_file}:EVENTS' imageset='my_spatial.fit' xcolumn='M_LAMBDA' ycolumn='XDSP_CORR'

evselect table='{evenli_file}:EVENTS' \
imageset='my_pi.fit' xcolumn='M_LAMBDA' ycolumn='PI'\
yimagemin=0 yimagemax=3000 \
expression='REGION({srcli_file}:RGS1_SRC1_SPATIAL,M_LAMBDA,XDSP_CORR)'

rgsimplot endispset='my_pi.fit' spatialset='my_spatial.fit' \
srcidlist='1' srclistset='{srcli_file}' \
device=/xs
''')

f.close()